In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import scipy as sp
from carbonplan_forest_risks import load, setup, plot, fit, utils, prepare, collect
import xarray as xr
from carbonplan_forest_risks.utils import get_store
import altair as alt
from carbonplan.data import cat
import rioxarray
import cartopy.crs as ccrs
import cartopy
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from carbonplan_styles.mpl import get_colormap
import warnings
warnings.filterwarnings('ignore')
import matplotlib
from carbonplan_data import utils

alt.themes.enable("carbonplan_light")

In [ ]:
alt.data_transformers.disable_max_rows()

In [ ]:
coarsen = 4
mask = (
    (
        load.nlcd(store="az", year=2001).sel(band=[41, 42, 43, 90]).sum("band")
        > 0.25
    )
    .astype("float")
    .coarsen(x=coarsen, y=coarsen, boundary="trim")
    .mean()
)

In [ ]:
historical_fire = xr.open_zarr(
    get_store(
        "carbonplan-forests", "risks/results/paper/fire_terraclimate.zarr"
    )
).load()
fire_mask = ~np.isnan(historical_fire.historical.isel(time=0).drop("time"))

In [ ]:
ds = (
    xr.open_zarr(
        get_store("carbonplan-forests", "risks/results/paper/fire_cmip.zarr")
    )
    .assign_coords({"x": mask.x, "y": mask.y})
    .where(fire_mask)
    .groupby("time.year")
    .sum()
    .where(fire_mask)
    .compute()
)

In [ ]:
future_maps = {}
future_maps["Fire"] = (
    ds.sel(year=slice("2080", "2099")).mean(dim="year").compute()
)

### load insects and drought


In [ ]:
for impact in ["drought", "insects"]:
    future_maps[impact.capitalize()] = (
        xr.open_zarr(
            get_store(
                "carbonplan-forests",
                "risks/results/paper/{}_cmip.zarr".format(impact),
            )
        )
        .assign_coords({"year": np.arange(1975, 2100, 10)})
        .sel(year=slice(2080, 2099))
        .mean(dim="year")
    )

In [ ]:
gcms = [
    ("MRI-ESM2-0", (0, 0)),
    ("MIROC-ES2L", (1, 0)),
    ("MPI-ESM1-2-LR", (2, 0)),
    ("ACCESS-ESM1-5", (3, 0)),
    ("ACCESS-CM2", (4, 0)),
    ("CanESM5-CanOE", (5, 0)),
]
titles = [
    "Burn area\n[fraction/year]",
    "Drought mortality\n[]",
    "Biotic agent mortality\n[]",
]

In [ ]:
scenario_dict = {}
for risk in ["Fire", "Insects", "Drought"]:
    scenario_dict[risk] = {}
    for scenario in ["ssp245", "ssp370", "ssp585"]:
        scenario_dict[risk][scenario] = (
            future_maps[risk]
            .probability.sel(scenario=scenario)
            .mean(dim="gcm")
            .drop("scenario")
        )

In [ ]:
plot_params = {
    "Fire": {
        "cmap": get_colormap("reds"),
        "var_lims": (0, 0.03),
        "label": "Burn area\n(fraction/year)",
        "panel": ["A", "B", "C"],
        "cbar_ylocation": 0.76,
    },
    "Drought": {
        "cmap": get_colormap("pinks"),
        "var_lims": (0, 4),
        "label": "Drought-related\nmortality (%/year)",
        "panel": ["D", "E", "F"],
        "cbar_ylocation": 0.4,
    },
    "Insects": {
        "cmap": get_colormap("blues"),
        "var_lims": (0, 0.8),
        "label": "Insect-related\nmortality (%/year)",
        "panel": ["G", "H", "I"],
        "cbar_ylocation": 0.04,
    },
}

In [ ]:
matplotlib.rc("font", family="sans-serif")
matplotlib.rc("font", serif="Helvetica Neue")
matplotlib.rc("text", usetex="false")
matplotlib.rcParams.update({"font.size": 14, "svg.fonttype": "none"})

In [ ]:
ssp_rename = {"ssp245": "SSP2-4.5", "ssp370": "SSP3-7.0", "ssp585": "SSP5-8.5"}

In [ ]:
state_borders, us_border = plot.cartopy_borders()
fig, axarr = plt.subplots(
    nrows=3,
    ncols=3,
    figsize=(15, 12),
    subplot_kw={"projection": plot.cartopy_proj_albers()},
)
from mpl_toolkits.axes_grid1 import make_axes_locatable

for row, impact in enumerate(["Fire", "Drought", "Insects"]):
    for column, scenario in enumerate(["ssp245", "ssp370", "ssp585"]):
        vmin, vmax = (
            plot_params[impact]["var_lims"][0],
            plot_params[impact]["var_lims"][1],
        )

        map_plot = scenario_dict[impact][scenario].plot.imshow(
            ax=axarr[row, column],
            cmap=plot_params[impact]["cmap"],
            vmin=vmin,
            vmax=vmax,
            add_colorbar=False,
        )

        plot.map_pretty(axarr[row, column], title=ssp_rename[scenario])
        axarr[row, column].text(
            0.12,
            1.05,
            plot_params[impact]["panel"][column],
            transform=axarr[row, column].transAxes,
            fontsize=18,
        )
    cax = fig.add_axes(
        [1.03, plot_params[impact]["cbar_ylocation"], 0.018, 0.12]
    )
    cax.text(
        0.5,
        -0.12,
        plot_params[impact]["var_lims"][0],
        transform=cax.transAxes,
        horizontalalignment="center",
    )
    cax.text(
        0.5,
        1.05,
        plot_params[impact]["var_lims"][1],
        transform=cax.transAxes,
        horizontalalignment="center",
    )
    cax.text(
        1.8,
        0.5,
        plot_params[impact]["label"],
        transform=cax.transAxes,
        verticalalignment="center",
        multialignment="center",
        rotation=-90,
    )
    cbar = fig.colorbar(map_plot, cax=cax, orientation="vertical")
    cbar.outline.set_visible(False)
    cbar.set_ticks([])
plt.tight_layout(pad=-6)
# for format_string in ["svg", "png"]:
#     fig.savefig(
#         "Figure-3." + format_string, format=format_string, bbox_inches="tight"
#     )